In [1]:
import torch

from training.inference import get_latents
from datasets import tqdm

def generate_evaluation_dataset(model, dataloader, name, chunking=True, chunk_size=256):
    model.mask_ratio = 0.0
    model.to("cuda")
    latents, labels = get_latents(dataloader, model, chunking=chunking, chunk_size=chunk_size)
    print("Saving...")
    for index, latent, label in tqdm(enumerate(zip(latents, labels))):
        directory = f"D:\\SongsDataset\\GTZAN\\latent_datasets\\{name}\\"
        label_name = directory + f"{index}_label.pt"
        latent_name = directory + f"{index}_data.pt"
        torch.save(latents, latent_name)
        torch.save(label, label_name)

In [2]:
from data.data_utils import GTZAN
from torch.utils.data import DataLoader

directory = "D:\\SongsDataset\\GTZAN\\genres_original\\"
GTZAN_dataset = GTZAN(directory)
GTZAN_dataloader = DataLoader(
    GTZAN_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=2,
    prefetch_factor=1
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [3]:
model = torch.load("E:\\Coding\\SongAnalyzer\\Analyzer\\src\\trained_models\\Myna-CLS-Sinusoidal\\Epoch-79.pt", weights_only=False)
generate_evaluation_dataset(model, GTZAN_dataloader, "Sinusoidal-Chunking-256", chunking=True, chunk_size=256)

  0%|          | 0/999 [00:13<?, ?it/s]


TypeError: expected np.ndarray (got Tensor)

In [ ]:
model = torch.load("E:\\Coding\\SongAnalyzer\\Analyzer\\src\\trained_models\\Myna-CLS-ALIBI\\Epoch-103.pt", weights_only=False)
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-256", chunking=True, chunk_size=256)

In [ ]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-512", chunking=True, chunk_size=512)

In [ ]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-1024", chunking=True, chunk_size=1024)

In [ ]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-2048", chunking=True, chunk_size=2048)
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-No-Chunking", chunking=False, chunk_size=131072)

In [ ]:
def linear_probing(dataloader):
